In [125]:
import sys
print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

from ieeg.navigate import channel_outlier_marker, trial_ieeg, crop_empty_data, \
    outliers_to_nan
from ieeg.io import raw_from_layout, get_data
from ieeg.timefreq.utils import crop_pad
from ieeg.timefreq import gamma
from ieeg.calc.scaling import rescale
import mne
import os
import numpy as np
from ieeg.calc.reshape import make_data_same
from ieeg.calc.stats import time_perm_cluster
from ieeg.viz.mri import gen_labels

from misc_functions import calculate_RTs, save_channels_to_file, save_sig_chans, load_sig_chans
import matplotlib.pyplot as plt
from collections import OrderedDict, defaultdict
import json
from misc_functions import load_sig_chans, channel_names_to_indices

import pandas as pd
from statsmodels.stats.multitest import multipletests

['c:\\Users\\jz421\\Desktop\\GlobalLocal', 'C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\L

In [2]:
# Initialize the outer dictionary.
subjects_electrodestoROIs_dict = {}

### make subjects rois to electrodes dict. Don't need to run this more than once.

In [ ]:
subjects = ['D0057','D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

# subjects = ['D0103'] #testing cuz d0065 being weird

for sub in subjects:
    # sub = 'D0059'
    task = 'GlobalLocal'
    output_name = "Response_fixationCrossBase_1sec_mirror"
    events = ["Response"]
    times = (-1,1.5)
    base_times = [-1,0]
    LAB_root = None
    channels = None
    full_trial_base = False


    if LAB_root is None:
        HOME = os.path.expanduser("~")
        if os.name == 'nt':  # windows
            LAB_root = os.path.join(HOME, "Box", "CoganLab")
        else:  # mac
            LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                    "CoganLab")

    layout = get_data(task, root=LAB_root)
    filt = raw_from_layout(layout.derivatives['derivatives/clean'], subject=sub,
                        extension='.edf', desc='clean', preload=False)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    good = crop_empty_data(filt)
    # %%

    print(f"good channels before dropping bads: {len(good.ch_names)}")
    print(f"filt channels before dropping bads: {len(filt.ch_names)}")

    good.info['bads'] = channel_outlier_marker(good, 3, 2)
    print("Bad channels in 'good':", good.info['bads'])

    filt.drop_channels(good.info['bads'])  # this has to come first cuz if you drop from good first, then good.info['bads'] is just empty
    good.drop_channels(good.info['bads'])

    print("Bad channels in 'good' after dropping once:", good.info['bads'])

    print(f"good channels after dropping bads: {len(good.ch_names)}")
    print(f"filt channels after dropping bads: {len(filt.ch_names)}")

    good.load_data()

    # If channels is None, use all channels
    if channels is None:
        channels = good.ch_names
    else:
        # Validate the provided channels
        invalid_channels = [ch for ch in channels if ch not in good.ch_names]
        if invalid_channels:
            raise ValueError(
                f"The following channels are not valid: {invalid_channels}")

        # Use only the specified channels
        good.pick_channels(channels)

    ch_type = filt.get_channel_types(only_data_chs=True)[0]
    good.set_eeg_reference(ref_channels="average", ch_type=ch_type)

    default_dict = gen_labels(good.info)
    
    # Create rawROI_dict for the subject
    rawROI_dict = defaultdict(list)
    for key, value in default_dict.items():
        rawROI_dict[value].append(key)
    rawROI_dict = dict(rawROI_dict)

    # Filter out keys containing "White-Matter"
    filtROI_dict = {key: value for key, value in rawROI_dict.items() if "White-Matter" not in key}

    # Store the dictionaries in the subjects dictionary
    subjects_electrodestoROIs_dict[sub] = {
        'default_dict': dict(default_dict),
        'rawROI_dict': dict(rawROI_dict),
        'filtROI_dict': dict(filtROI_dict)
    }


# # Save to a JSON file. Uncomment when actually running.
filename = 'subjects_electrodestoROIs_dict.json'
with open(filename, 'w') as file:
    json.dump(subjects_electrodestoROIs_dict, file, indent=4)

print(f"Saved subjects_dict to {filename}")

### load subjects electrodes to rois dict

In [3]:
# Load from a JSON file
filename = 'subjects_electrodestoROIs_dict.json'

with open(filename, 'r') as file:
    subjects_electrodestoROIs_dict = json.load(file)

print(f"Loaded data from {filename}")

Loaded data from subjects_electrodestoROIs_dict.json


### load high gamma data so we can do roi analysis on it
once we have more subjects, turn this into a function and loop over all subjects.  
this code is a crime against humanity

In [4]:
import os
import mne
import numpy as np


def load_mne_objects(sub, output_name, task, LAB_root=None):
    """
    Load MNE objects for a given subject and output name.

    Parameters:
    - sub (str): Subject identifier.
    - output_name (str): Output name used in the file naming.
    - task (str): Task identifier.
    - LAB_root (str, optional): Root directory for the lab. If None, it will be determined based on the OS.

    Returns:
    A dictionary containing loaded MNE objects.
    """

    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)
    
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Define file paths
    HG_ev1_file = f'{save_dir}/{sub}_{output_name}_HG_ev1-epo.fif'
    HG_base_file = f'{save_dir}/{sub}_{output_name}_HG_base-epo.fif'
    HG_ev1_rescaled_file = f'{save_dir}/{sub}_{output_name}_HG_ev1_rescaled-epo.fif'

    # Load the objects
    HG_ev1 = mne.read_epochs(HG_ev1_file)
    HG_base = mne.read_epochs(HG_base_file)
    HG_ev1_rescaled = mne.read_epochs(HG_ev1_rescaled_file)
    HG_ev1_evoke = HG_ev1.average(method=lambda x: np.nanmean(x, axis=0))
    HG_ev1_evoke_rescaled = HG_ev1_rescaled.average(method=lambda x: np.nanmean(x, axis=0))

    return {
        'HG_ev1': HG_ev1,
        'HG_base': HG_base,
        'HG_ev1_rescaled': HG_ev1_rescaled,
        'HG_ev1_evoke': HG_ev1_evoke,
        'HG_ev1_evoke_rescaled': HG_ev1_evoke_rescaled
    }

# Example usage
# sub = 'D0057'
# output_name = "Stimulus_i25and75_fixationCrossBase_1sec_mirror"
# task = 'GlobalLocal'
loaded_objects_D0057_i = load_mne_objects('D0057', "Stimulus_c25_fixationCrossBase_1sec_mirror", 'GlobalLocal')
loaded_objects_D0057_c = load_mne_objects('D0057', "Stimulus_c75_fixationCrossBase_1sec_mirror", 'GlobalLocal')

# Access the objects
HG_ev1_D0057_i = loaded_objects_D0057_i['HG_ev1']
HG_base_D0057_i = loaded_objects_D0057_i['HG_base']
HG_ev1_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_rescaled']
HG_ev1_evoke_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_i = loaded_objects_D0057_i['HG_ev1_evoke_rescaled']

HG_ev1_D0057_c = loaded_objects_D0057_c['HG_ev1']
HG_base_D0057_c = loaded_objects_D0057_c['HG_base']
HG_ev1_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_rescaled']
HG_ev1_evoke_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke']
HG_ev1_evoke_rescaled_D0057_c = loaded_objects_D0057_c['HG_ev1_evoke_rescaled']

Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
168 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_c25_fixationCrossBase_1sec_mirror_HG_ev1_rescaled-epo.fif ...
    Found the data of interest:
        t =   -1000.

load accuracy arrays so we can filter by only accurate trials  
turn this into a single function later

In [5]:
# Directory where your .npy files are saved
npy_directory = r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\accArrays'  # Replace with your directory path

# Dictionary to hold the data
acc_array = {}

# Iterate over each file in the directory
for file in os.listdir(npy_directory):
    if file.endswith('.npy'):
        # Construct the full file path
        file_path = os.path.join(npy_directory, file)
        # Load the numpy array from the file
        acc_array[file.split('_')[0]] = np.load(file_path)

# Now you have a dictionary where each key is the subject ID
# and the value is the numpy array of accuracies for that subject.

In [6]:
def add_accuracy_to_epochs(epochs, accuracy_array):
    """
    Adds accuracy data from accuracy_array to the metadata of epochs.
    Assumes the order of trials in accuracy_array matches the order in epochs.
    """
    if epochs.metadata is None:
        # Create a new DataFrame if no metadata exists
        epochs.metadata = pd.DataFrame(index=range(len(epochs)))
    
    # Ensure the accuracy_array length matches the number of epochs
    assert len(accuracy_array) == len(epochs), "Mismatch in number of trials and accuracy data length."
    
    # Add the accuracy array as a new column in the metadata
    epochs.metadata['accuracy'] = accuracy_array

    # Reset the index to ensure it's sequential starting from 0
    epochs.metadata.reset_index(drop=True, inplace=True)
    
    return epochs


okay so we need to grab the event timings from HG_ev1_rescaled.events in samples, and convert to seconds.  
Although keep in mind these will be early by however long the event starts before the stimulus onset, when compared to the combinedData.csv.  
Then, load in combinedData.csv for this subject and grab the trialCounts that have stimOnsets that match up with the events.

In [7]:
combined_data = pd.read_csv(r'C:\Users\jz421\Box\CoganLab\D_Data\GlobalLocal\combinedData.csv')

### load evoked and stuff for all subjects in a dictionary

In [75]:
import pandas as pd
import os
import mne
import numpy as np

def create_subjects_mne_objects_dict(subjects, output_names_conditions, task, combined_data, LAB_root=None):
    """
    Adjusted to handle multiple conditions per output name, with multiple condition columns.

    Parameters:
    - subjects: List of subject IDs.
    - output_names_conditions: Dictionary where keys are output names and values are dictionaries
        of condition column names and their required values.
    - task: Task identifier.
    - combined_data: DataFrame with combined behavioral and trial information.
    - LAB_root: Root directory for data (optional).
    """
    subjects_mne_objects = {}

    for sub in subjects:
        print(f"Loading data for subject: {sub}")
        sub_mne_objects = {}
        for output_name, conditions in output_names_conditions.items():
            print(f"  Loading output: {output_name} with conditions: {conditions}")
            
            # Build the filtering condition
            sub_without_zeroes = "D" + sub[1:].lstrip('0') 
            condition_filter = (combined_data['subject_ID'] == sub_without_zeroes) # this indexes using the subject without zeroes in the name. Confusing. I know.
            
            for condition_column, condition_value in conditions.items():
                if isinstance(condition_value, list):
                    # If the condition needs to match any value in a list
                    condition_filter &= combined_data[condition_column].isin(condition_value)
                else:
                    # If the condition is a single value
                    condition_filter &= (combined_data[condition_column] == condition_value)
            
            # Filter combinedData for the specific subject and conditions
            subject_condition_data = combined_data[condition_filter]
            
            # Load MNE objects and update with accuracy data
            mne_objects = load_mne_objects(sub, output_name, task, LAB_root)
            
            if sub in acc_array:
                trial_counts = subject_condition_data['trialCount'].values.astype(int)
                accuracy_data = [acc_array[sub][i-1] for i in trial_counts if i-1 < len(acc_array[sub])] # Subtract 1 here for zero-based indexing in acc array.
                
                # Now pass trial_counts along with accuracy_data
                mne_objects['HG_ev1_rescaled'] = add_accuracy_to_epochs(mne_objects['HG_ev1_rescaled'], accuracy_data)

            sub_mne_objects[output_name] = mne_objects
        subjects_mne_objects[sub] = sub_mne_objects

    return subjects_mne_objects


# # example of how to use this with multiple conditions, even matching any value in a list. Although I only ever have two conditions of a type so not super necessary.
# # make sure to use the correct column names and values that match with what combinedData uses.
# output_names_conditions = {
#     "Stimulus_c25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#         "switchType": ["s1", "s2"]  # Example where switchType needs to match any value in the list
#     },
#     "Stimulus_i25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#         "switchType": "s"
#     }
# }

subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0071', 'D0077', 'D0090', 'D0100', 'D0102', 'D0103']
# # congruency
# output_names = ["Stimulus_c25and75_fixationCrossBase_1sec_mirror", "Stimulus_i25and75_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_c25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "c",
#     },
#     "Stimulus_i25and75_fixationCrossBase_1sec_mirror": {
#         "congruency": "i",
#     }
# }

# # switch
# output_names = ["Stimulus_r25and75_fixationCrossBase_1sec_mirror", "Stimulus_s25and75_fixationCrossBase_1sec_mirror"]
# output_names_conditions = {
#     "Stimulus_r25and75_fixationCrossBase_1sec_mirror": {
#         "switchType": "r",
#     },
#     "Stimulus_s25and75_fixationCrossBase_1sec_mirror": {
#         "switchType": "s",
#     }
# }

#  ir vs is
output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
output_names_conditions = {
    "Stimulus_ir_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "r"
    },
    "Stimulus_is_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "s"
    }
}

#  cr vs cs
output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]
output_names_conditions = {
    "Stimulus_cr_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "r"
    },
    "Stimulus_cs_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "s"
    }
}

#  is vs cs
output_names = ["Stimulus_cs_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror"]
output_names_conditions = {
    "Stimulus_cs_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "s"
    },
    "Stimulus_is_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "s"
    }
}

#  ir vs cr
output_names = ["Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_ir_fixationCrossBase_1sec_mirror"]
output_names_conditions = {
    "Stimulus_cr_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "r"
    },
    "Stimulus_ir_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "r"
    }
}

# all interaction effects (run this with the anova code. Ugh make everything more modular later.)
output_names = ["Stimulus_ir_fixationCrossBase_1sec_mirror", "Stimulus_is_fixationCrossBase_1sec_mirror", "Stimulus_cr_fixationCrossBase_1sec_mirror", "Stimulus_cs_fixationCrossBase_1sec_mirror"]

output_names_conditions = {
    "Stimulus_ir_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "r"
    },
    "Stimulus_is_fixationCrossBase_1sec_mirror": {
        "congruency": "i",
        "switchType": "s"
    },
    "Stimulus_cr_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "r"
    },
    "Stimulus_cs_fixationCrossBase_1sec_mirror": {
        "congruency": "c",
        "switchType": "s"
    }
}

task='GlobalLocal'

# Assuming 'combined_data' is your DataFrame and 'subjects' is your list of subject IDs
subjects_mne_objects = create_subjects_mne_objects_dict(subjects, output_names_conditions, task="GlobalLocal", combined_data=combined_data)

Loading data for subject: D0057
  Loading output: Stimulus_cr_fixationCrossBase_1sec_mirror with conditions: {'congruency': 'c', 'switchType': 'r'}
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_cr_fixationCrossBase_1sec_mirror_HG_ev1-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...    1500.00 ms
        0 CTF compensation matrices available
Not setting metadata
121 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs\D0057\D0057_Stimulus_cr_fixationCrossBase_1sec_mirror_HG_base-epo.fif ...
    Found the data of interest:
        t =   -1000.00 ...       0.00 ms
        0 CTF compensation matrices available
Not setting metadata
448 matching events found
No baseline correction applied
0 projection items activated
Reading C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\fre

old way to create mne objects dict without accuracy indexing. Delete once filtering by accuracy works.

In [ ]:
# import os
# import mne
# import numpy as np

# def create_subjects_mne_objects_dict(subjects, output_names, task, LAB_root=None):
#     subjects_mne_objects = {}

#     for sub in subjects:
#         print(f"Loading data for subject: {sub}")  # Debugging print
#         sub_mne_objects = {}
#         for output_name in output_names:
#             print(f"  Loading output: {output_name}")  # Debugging print
#             mne_objects = load_mne_objects(sub, output_name, task, LAB_root)

#             # Debugging prints for data shapes
#             print(f"    HG_ev1 shape: {mne_objects['HG_ev1'].get_data().shape}")
#             print(f"    HG_base shape: {mne_objects['HG_base'].get_data().shape}")
#             print(f"    HG_ev1_rescaled shape: {mne_objects['HG_ev1_rescaled'].get_data().shape}")
#             print(f"    HG_ev1_evoke shape: {mne_objects['HG_ev1_evoke'].data.shape}")
#             print(f"    HG_ev1_evoke_rescaled shape: {mne_objects['HG_ev1_evoke_rescaled'].data.shape}")

#             # NEW code 2/6 for accuracy metadata
#             # Assuming each subject's accuracy array is loaded into acc_array dict
#             if sub in acc_array:
#                 accuracy_data = acc_array[sub]
#                 #update hg ev1 rescaled with accuracy data.
#                 mne_objects['HG_ev1_rescaled'] = add_accuracy_to_epochs(mne_objects['HG_ev1_rescaled'], accuracy_data)

#             sub_mne_objects[output_name] = mne_objects
#         subjects_mne_objects[sub] = sub_mne_objects

#     return subjects_mne_objects

# # Example usage
# subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']
# output_names = ["Stimulus_c25and75_fixationCrossBase_1sec_mirror", "Stimulus_i25and75_fixationCrossBase_1sec_mirror"]
# task = 'GlobalLocal'

# subjects_mne_objects = create_subjects_mne_objects_dict(subjects, output_names, task)

### load stimulus significant channels. Compare ROI electrodes in next cell to these to see if they're included.

maybe do response significant channels too/instead?

In [40]:
def get_sig_chans(sub, task, LAB_root=None):
    # Determine LAB_root based on the operating system
    if LAB_root is None:
        HOME = os.path.expanduser("~")
        LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

    # Get data layout
    layout = get_data(task, root=LAB_root)
    save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs', sub)

    stim_filename = f'{save_dir}\\sig_chans_{sub}_Stimulus_fixationCrossBase_1sec_mirror.json'
    stim_sig_chans = load_sig_chans(stim_filename)
    return stim_sig_chans


# Initialize an empty dictionary to store significant channels per subject
sig_chans_per_subject = {}

# Populate the dictionary using get_sig_chans for each subject
for sub in subjects:
    sig_chans_per_subject[sub] = get_sig_chans(sub, 'GlobalLocal')

# Now sig_chans_per_subject dictionary is populated with significant channels for each subject

Loaded significant channels for subject D0057
Loaded significant channels for subject D0059
Loaded significant channels for subject D0063
Loaded significant channels for subject D0065
Loaded significant channels for subject D0071
Loaded significant channels for subject D0077
Loaded significant channels for subject D0090
Loaded significant channels for subject D0100
Loaded significant channels for subject D0102
Loaded significant channels for subject D0103


### get the significant electrodes across subjects for each ROI of interest

dlPFC based on Yamagishi et al 2016 definition is G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup
ACC based on Destrieux et al 2010 definition is G_and_S_cingul-Ant

In [41]:
def filter_electrodes_by_roi(subjects_electrodes_dict, sig_chans_per_subject, roi_list):
    """
    Filters electrodes based on specified ROIs and returns significant electrodes for each subject.

    Args:
    subjects_electrodes_dict (dict): A dictionary with subjects as keys and electrode-to-ROI mappings as values.
    sig_chans_per_subject (dict): A dictionary with subjects as keys and lists of significant channels as values.
    roi_list (list): A list of ROIs to filter electrodes.

    Returns:
    dict: A dictionary with subjects as keys and lists of significant electrodes in specified ROIs as values.
    """
    filtered_electrodes_per_subject = {}

    for sub, electrodes_dict in subjects_electrodes_dict.items():
        filtered = {key: value for key, value in electrodes_dict['filtROI_dict'].items() 
                    if any(roi in key for roi in roi_list)}

        # Aggregate electrodes into a list for each subject
        filtered_electrodes = []
        for electrodes in filtered.values():
            filtered_electrodes.extend(electrodes)

        filtered_electrodes_per_subject[sub] = filtered_electrodes
        print(f'For subject {sub}, {", ".join(roi_list)} electrodes are: {filtered_electrodes}')

    # Now filter for significant electrodes
    sig_filtered_electrodes_per_subject = {}

    for sub, filtered_electrodes in filtered_electrodes_per_subject.items():
        # Retrieve the list of significant channels for the subject
        sig_chans = sig_chans_per_subject.get(sub, [])

        # Find the intersection of filtered electrodes and significant channels for the subject
        sig_filtered_electrodes = [elec for elec in filtered_electrodes if elec in sig_chans]

        # Store the significant filtered electrodes for the subject
        sig_filtered_electrodes_per_subject[sub] = sig_filtered_electrodes
        print(f"Subject {sub} significant {', '.join(roi_list)} electrodes: {sig_filtered_electrodes}")

    return filtered_electrodes_per_subject, sig_filtered_electrodes_per_subject

# Example usage:
dlpfc_rois = ["G_front_middle", "G_front_sup", "S_front_inf", "S_front_middle", "S_front_sup"] #dorsolateral prefrontal cortex
acc_rois = ["G_and_S_cingul-Ant", "G_and_S_cingul-Mid-Ant"] #anterior cingulate cortex
parietal_rois = ["G_parietal_sup", "S_intrapariet_and_P_trans", "G_pariet_inf-Angular", "G_pariet_inf-Supramar"] #superior parietal lobule, intraparietal sulcus, and inferior parietal lobule (split into angular gyrus and supramarginal gyrus)
dlpfc_electrodes_per_subject, sig_dlpfc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, dlpfc_rois)
acc_electrodes_per_subject, sig_acc_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, acc_rois)
parietal_electrodes_per_subject, sig_parietal_electrodes_per_subject = filter_electrodes_by_roi(subjects_electrodestoROIs_dict, sig_chans_per_subject, parietal_rois)

For subject D0103, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LFAM8', 'LFAM9', 'LAI13', 'LAI14', 'LFAM15']
For subject D0057, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RAI12', 'RAI13', 'RAI15', 'RAI16', 'RPI14', 'RAMF10', 'RAMF11', 'RAMF12']
For subject D0059, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16']
For subject D0063, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['LASF10', 'LASF14', 'LASF15', 'LASF16', 'LMSF5', 'LMSF6', 'LMSF12', 'LPSF10', 'LPSF12', 'RAI10', 'RAI11', 'RAI16', 'RAMF11', 'RAMF12', 'RAMF13', 'RMMF13', 'RMMF14', 'RASF15', 'RMSF8', 'RMSF9', 'RMSF10', 'RMSF7', 'RAMF8', 'RAMF9', 'RAMF10', 'RMMF9', 'RMMF10']
For subject D0065, G_front_middle, G_front_sup, S_front_inf, S_front_middle, S_front_sup electrodes are: ['RASF13', 'RASF14', 'RASF15', 'RMSF11', 'RMSF12', '

get total number of electrodes (make this modular with roi later once everything works)

In [42]:
total_entries = 0
for sub in sig_dlpfc_electrodes_per_subject:
    # Since each subject's entry is a list, directly add its length
    total_entries += len(sig_dlpfc_electrodes_per_subject[sub])

print("Total number of sig dlpfc electrodes across all subjects:", total_entries)

total_entries = 0
for sub in sig_acc_electrodes_per_subject:
    # Since each subject's entry is a list, directly add its length
    total_entries += len(sig_acc_electrodes_per_subject[sub])

print("Total number of sig acc electrodes across all subjects:", total_entries)

total_entries = 0
for sub in sig_parietal_electrodes_per_subject:
    # Since each subject's entry is a list, directly add its length
    total_entries += len(sig_parietal_electrodes_per_subject[sub])

print("Total number of sig parietal electrodes across all subjects:", total_entries)

Total number of sig dlpfc electrodes across all subjects: 22
Total number of sig acc electrodes across all subjects: 6
Total number of sig parietal electrodes across all subjects: 13


In [43]:
sig_dlpfc_electrodes_per_subject

{'D0103': ['LFAM8', 'LFAM9', 'LAI13', 'LAI14'],
 'D0057': ['RPI14'],
 'D0059': ['LMMF9', 'LMMF11', 'LMMF12', 'LPSF16'],
 'D0063': ['LMSF5', 'LPSF12', 'RAMF12', 'RMMF13', 'RMMF14', 'RMMF10'],
 'D0065': ['RASF14'],
 'D0069': [],
 'D0071': ['RIA11', 'RIA12', 'RIA14', 'RIA16'],
 'D0077': [],
 'D0090': ['RIA12'],
 'D0094': [],
 'D0100': [],
 'D0102': ['RFAM15']}

In [44]:
sig_acc_electrodes_per_subject

{'D0103': ['LFAM3'],
 'D0057': [],
 'D0059': [],
 'D0063': ['LMSF2', 'LMSF4', 'RMSF3', 'RMSF4'],
 'D0065': [],
 'D0069': [],
 'D0071': [],
 'D0077': [],
 'D0090': [],
 'D0094': [],
 'D0100': [],
 'D0102': ['RFMM1']}

In [45]:
sig_parietal_electrodes_per_subject

{'D0103': [],
 'D0057': ['RPIP11', 'RPIP12', 'RPIP14'],
 'D0059': ['LMPF11', 'LMPF12', 'LMPF14'],
 'D0063': [],
 'D0065': [],
 'D0069': [],
 'D0071': ['RTPS9'],
 'D0077': ['ROAS14', 'ROPS10', 'ROPS12', 'RPAS7', 'ROAS7', 'ROAS11'],
 'D0090': [],
 'D0094': [],
 'D0100': [],
 'D0102': []}

### do stats

current approach is to run time_perm_cluster on significant dlpfc electrodes for each subject, comparing congruent and incongruent conditions. Then, average p-values across all subjects. Discuss this with Greg, probably wrong approach.

**1/23 new approach is to average across all trials for sig dlpfc electrodes, comparing incongruent and congruent conditions. Then, run stats on this new avg electrode value x time array.

Also, I'm using HG_ev1_rescaled instead of HG_ev1 to compare congruent and incongruent, so that they're normalized with a common baseline. I think this is better than comparing the raw HG traces directly.

### this is 1/23 new approach of avg across trials first

do stats and plotting together. Stats needs trial avg data, plotting just needs congruent_data without trial averaging (initially at least)  
this code is so bad right now, turn into a function later  

trialAvg is for the time perm cluster stats  
timeAvg_firstHalfSecond_firstHalfSecond_firstHalfSecond_firstHalfSecond_firstHalfSecond is for the window stats (not sure if this is even right)  

here, also only average across the accurate trials



In [64]:
def filter_and_average_epochs(epochs, start_idx, end_idx, accuracy_column='accuracy'):
    """
    Calculates trial averages for accurate trials and time averages with inaccurate trials marked as NaNs.

    Parameters:
    - epochs: MNE Epochs object with accuracy metadata.
    - start_idx: Start index for time averaging.
    - end_idx: End index for time averaging.
    - accuracy_column: Name of the column in the metadata that contains accuracy data.

    Returns:
    - trial_avg_data: Trial-averaged data across accurate trials.
    - time_avg_data: Time-averaged data with inaccurate trials marked as NaNs.
    """
    # Separate accurate and all trials data
    accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
    all_epochs_data = epochs.get_data().copy()

    # Mark inaccurate trials as NaNs in the all_epochs_data
    inaccurate_indices = epochs.metadata[accuracy_column] != 1.0
    all_epochs_data[inaccurate_indices, :, :] = np.nan

    # Calculate trial average for accurate trials
    trial_avg_data = np.nanmean(accurate_epochs_data, axis=0)

    # Calculate time average within the specified window
    time_avg_data = np.nanmean(all_epochs_data[:, :, start_idx:end_idx], axis=2)

    return trial_avg_data, time_avg_data


turn these into dictionaries instead of a bunch of variables later

In [82]:
len(output_0_data_timeAvg_firstHalfSecond_list['dlpfc'])

8

In [134]:
# Initialize a dictionary to hold mappings
overall_electrode_mapping = []

# Initialize a dictionary to hold mappings for each ROI
electrode_mapping_per_roi = {
    'dlpfc': [],
    'acc': [],
    'parietal': []
}

# Initialize lists for storing data
output_0_data_trialAvg_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_1_data_trialAvg_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_0_data_timeAvg_firstHalfSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_1_data_timeAvg_firstHalfSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_0_data_timeAvg_secondHalfSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_1_data_timeAvg_secondHalfSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_0_data_timeAvg_fullSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}
output_1_data_timeAvg_fullSecond_list = {'dlpfc': [], 'acc': [], 'parietal': []}

# Time windows
start_idx_firstHalfSecond, end_idx_firstHalfSecond = 2048, 3072
start_idx_secondHalfSecond, end_idx_secondHalfSecond = 3072, 4096
start_idx_fullSecond, end_idx_fullSecond = 2048, 4096


for sub in subjects:
    for roi in ['dlpfc', 'acc', 'parietal']:
        # Determine the significant electrodes for the current roi
        if roi == 'dlpfc':
            sig_electrodes = sig_dlpfc_electrodes_per_subject.get(sub, [])
        elif roi == 'acc':
            sig_electrodes = sig_acc_electrodes_per_subject.get(sub, [])
        else:  # parietal
            sig_electrodes = sig_parietal_electrodes_per_subject.get(sub, [])
        
        # Skip this roi for the current subject if no significant electrodes are present
        if not sig_electrodes:
            continue
        for electrode in sig_electrodes:
            # For each significant electrode, append a tuple to the mapping list
            # Tuple format: (Subject ID, ROI, Electrode Name, Index in List)
            # The index can be the current length of the list before appending
            index = len(overall_electrode_mapping)
            overall_electrode_mapping.append((sub, roi, electrode, index))  

            # For each significant electrode, append a tuple to the mapping list of the corresponding ROI
            # Tuple format: (Subject ID, Electrode Name, Index in List for this ROI)
            index = len(electrode_mapping_per_roi[roi])  # Get the current length of the list for this ROI
            electrode_mapping_per_roi[roi].append((sub, electrode, index))
            
        # Load trial-level data for the current condition and pick significant electrodes
        output_0_epochs = subjects_mne_objects[sub][output_names[0]]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
        output_1_epochs = subjects_mne_objects[sub][output_names[1]]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)

        # Calculate averages for each time window
        trial_avg_0, time_avg_0_firstHalfSecond = filter_and_average_epochs(output_0_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
        trial_avg_1, time_avg_1_firstHalfSecond = filter_and_average_epochs(output_1_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
        _, time_avg_0_secondHalfSecond = filter_and_average_epochs(output_0_epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
        _, time_avg_1_secondHalfSecond = filter_and_average_epochs(output_1_epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
        _, time_avg_0_fullSecond = filter_and_average_epochs(output_0_epochs, start_idx_fullSecond, end_idx_fullSecond)
        _, time_avg_1_fullSecond = filter_and_average_epochs(output_1_epochs, start_idx_fullSecond, end_idx_fullSecond)

        # Append the results to their respective lists
        output_0_data_trialAvg_list[roi].append(trial_avg_0)
        output_1_data_trialAvg_list[roi].append(trial_avg_1)
        output_0_data_timeAvg_firstHalfSecond_list[roi].append(time_avg_0_firstHalfSecond)
        output_1_data_timeAvg_firstHalfSecond_list[roi].append(time_avg_1_firstHalfSecond)
        output_0_data_timeAvg_secondHalfSecond_list[roi].append(time_avg_0_secondHalfSecond)
        output_1_data_timeAvg_secondHalfSecond_list[roi].append(time_avg_1_secondHalfSecond)
        output_0_data_timeAvg_fullSecond_list[roi].append(time_avg_0_fullSecond)
        output_1_data_timeAvg_fullSecond_list[roi].append(time_avg_1_fullSecond)

# After collecting all data, concatenate across subjects for each roi and condition
concatenated_trialAvg_data = {}
concatenated_timeAvg_firstHalfSecond_data = {}
concatenated_timeAvg_secondHalfSecond_data = {}
concatenated_timeAvg_fullSecond_data = {}

for roi in ['dlpfc', 'acc', 'parietal']:
    concatenated_trialAvg_data[roi] = {
        'output_0': np.concatenate(output_0_data_trialAvg_list[roi], axis=0),
        'output_1': np.concatenate(output_1_data_trialAvg_list[roi], axis=0)
    }
    # concatenated_timeAvg_firstHalfSecond_data[roi] = {
    #     'output_0': np.concatenate(output_0_data_timeAvg_firstHalfSecond_list[roi], axis=1),
    #     'output_1': np.concatenate(output_1_data_timeAvg_firstHalfSecond_list[roi], axis=1)
    # }
    # concatenated_timeAvg_secondHalfSecond_data[roi] = {
    #     'output_0': np.concatenate(output_0_data_timeAvg_secondHalfSecond_list[roi], axis=1),
    #     'output_1': np.concatenate(output_1_data_timeAvg_secondHalfSecond_list[roi], axis=1)
    # }
    # concatenated_timeAvg_fullSecond_data[roi] = {
    #     'output_0': np.concatenate(output_0_data_timeAvg_fullSecond_list[roi], axis=1),
    #     'output_1': np.concatenate(output_1_data_timeAvg_fullSecond_list[roi], axis=1)
    # }


# Calculate mean and SEM across electrodes for all time windows and rois
overall_averages = {}
overall_sems = {}
for roi in ['dlpfc', 'acc', 'parietal']:
    overall_averages[roi] = {}
    overall_sems[roi] = {}
    for output in ['output_0', 'output_1']:
        trialAvg_data = concatenated_trialAvg_data[roi][output]
        overall_averages[roi][output] = np.nanmean(trialAvg_data, axis=0)
        overall_sems[roi][output] = np.std(trialAvg_data, axis=0, ddof=1) / np.sqrt(trialAvg_data.shape[0])



time_perm_cluster_results = {}
for roi in ['dlpfc', 'acc', 'parietal']:
    time_perm_cluster_results[roi] = time_perm_cluster(
        concatenated_trialAvg_data[roi]['output_0'],
        concatenated_trialAvg_data[roi]['output_1'], 0.05, n_jobs=6
    )

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:27: RuntimeWarning: Mean of empty slice
  time_avg_data = np.nanmean(all_epochs_data[:, :, start_idx:end_idx], axis=2)
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy 

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


...

C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy 

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy 

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  all_epochs_data = epochs.get_data().copy()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:16: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy explicitly to avoid this warning
  accurate_epochs_data = epochs[epochs.metadata[accuracy_column] == 1.0].get_data()
C:\Users\jz421\AppData\Local\Temp\ipykernel_16716\2439027647.py:17: FutureWarning: The current default of copy=False will change to copy=True in 1.7. Set the value of copy 

In [ ]:
# import numpy as np

# # Initialize lists to store data
# output_0_data_trialAvg_list_dlpfc = []
# output_1_data_trialAvg_list_dlpfc = []
# output_0_data_trialAvg_list_acc = []
# output_1_data_trialAvg_list_acc = []
# output_0_data_trialAvg_list_parietal = []
# output_1_data_trialAvg_list_parietal = []

# # Initialize lists to store time-averaged data
# output_0_data_timeAvg_firstHalfSecond_list_dlpfc = []
# output_1_data_timeAvg_firstHalfSecond_list_dlpfc = []
# output_0_data_timeAvg_firstHalfSecond_list_acc = []
# output_1_data_timeAvg_firstHalfSecond_list_acc = []
# output_0_data_timeAvg_firstHalfSecond_list_parietal = []
# output_1_data_timeAvg_firstHalfSecond_list_parietal = []

# # Initialize additional lists for the second half-second time-averaged data
# output_0_data_timeAvg_secondHalfSecond_list_dlpfc = []
# output_1_data_timeAvg_secondHalfSecond_list_dlpfc = []
# output_0_data_timeAvg_secondHalfSecond_list_acc = []
# output_1_data_timeAvg_secondHalfSecond_list_acc = []
# output_0_data_timeAvg_secondHalfSecond_list_parietal = []
# output_1_data_timeAvg_secondHalfSecond_list_parietal = []

# start_idx_firstHalfSecond, end_idx_firstHalfSecond = 2048, 3072  # Time window from 0 to 0.5 seconds
# start_idx_secondHalfSecond, end_idx_secondHalfSecond = 3072, 4096 # time window from 0.5 to 1 seconds
# start_idx_fullSecond, end_idx_fullSecond = 2048, 4096

# # subjects = ['D0057', 'D0059', 'D0063', 'D0065', 'D0069', 'D0071', 'D0077', 'D0090', 'D0094', 'D0100', 'D0102', 'D0103']

# for sub in subjects:
#     # Skip this subject if no dlpfc electrodes
#     if not sig_dlpfc_electrodes_per_subject[sub]:
#         continue

#     # Load trial-level data for each condition and pick significant DLPFC electrodes
#     output_0_epochs = subjects_mne_objects[sub][output_names[0]]['HG_ev1_rescaled'].copy().pick_channels(sig_dlpfc_electrodes_per_subject[sub])
#     output_1_epochs = subjects_mne_objects[sub][output_names[1]]['HG_ev1_rescaled'].copy().pick_channels(sig_dlpfc_electrodes_per_subject[sub])

#     # Use the new function to filter based on accuracy and calculate averages
#     trial_avg_0_dlpfc, time_avg_0_firstHalfSecond_dlpfc = filter_and_average_epochs(output_0_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
#     trial_avg_1_dlpfc, time_avg_1_firstHalfSecond_dlpfc = filter_and_average_epochs(output_1_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)

#     # Append the results to the respective lists
#     output_0_data_trialAvg_list_dlpfc.append(trial_avg_0_dlpfc)
#     output_1_data_trialAvg_list_dlpfc.append(trial_avg_1_dlpfc)
#     output_0_data_timeAvg_firstHalfSecond_list_dlpfc.append(time_avg_0_firstHalfSecond_dlpfc)
#     output_1_data_timeAvg_firstHalfSecond_list_dlpfc.append(time_avg_1_firstHalfSecond_dlpfc)

# # do for acc
# for sub in subjects:
#     # Skip this subject if no acc electrodes
#     if not sig_acc_electrodes_per_subject[sub]:
#         continue

#     # Load trial-level data for each condition and pick significant DLPFC electrodes
#     output_0_epochs = subjects_mne_objects[sub][output_names[0]]['HG_ev1_rescaled'].copy().pick_channels(sig_acc_electrodes_per_subject[sub])
#     output_1_epochs = subjects_mne_objects[sub][output_names[1]]['HG_ev1_rescaled'].copy().pick_channels(sig_acc_electrodes_per_subject[sub])

#     # Use the new function to filter based on accuracy and calculate averages
#     trial_avg_0_acc, time_avg_0_firstHalfSecond_acc = filter_and_average_epochs(output_0_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
#     trial_avg_1_acc, time_avg_1_firstHalfSecond_acc = filter_and_average_epochs(output_1_epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)

#     # Append the results to the respective lists
#     output_0_data_trialAvg_list_acc.append(trial_avg_0_acc)
#     output_1_data_trialAvg_list_acc.append(trial_avg_1_acc)
#     output_0_data_timeAvg_firstHalfSecond_list_acc.append(time_avg_0_firstHalfSecond_acc)
#     output_1_data_timeAvg_firstHalfSecond_list_acc.append(time_avg_1_firstHalfSecond_acc)
    
# # #do for parietal
# # for sub in subjects:
# #     # Skip this subject if no dlpfc electrodes
# #     if not sig_parietal_electrodes_per_subject[sub]:
# #         continue

# #     # Load trial-level data for each condition and pick significant DLPFC electrodes
# #     output_0_epochs = subjects_mne_objects[sub][output_names[0]]['HG_ev1_rescaled'].copy().pick_channels(sig_parietal_electrodes_per_subject[sub])
# #     output_1_epochs = subjects_mne_objects[sub][output_names[1]]['HG_ev1_rescaled'].copy().pick_channels(sig_parietal_electrodes_per_subject[sub])

# #     # Use the new function to filter based on accuracy and calculate averages
# #     trial_avg_0_parietal, time_avg_0_parietal = filter_and_average_epochs(output_0_epochs, start_idx, end_idx)
# #     trial_avg_1_parietal, time_avg_1_parietal = filter_and_average_epochs(output_1_epochs, start_idx, end_idx)

# #     # Append the results to the respective lists
# #     output_0_data_trialAvg_list_parietal.append(trial_avg_0_parietal)
# #     output_1_data_trialAvg_list_parietal.append(trial_avg_1_parietal)
# #     output_0_data_timeAvg_list_parietal.append(time_avg_0_parietal)
# #     output_1_data_timeAvg_list_parietal.append(time_avg_1_parietal)


# # # below functions only go this far, keep this code
# # # Concatenate data across all electrodes
# # output_0_data_trialAvg_dlpfc = np.concatenate(output_0_data_trialAvg_list_dlpfc, axis=0)
# # output_1_data_trialAvg_dlpfc = np.concatenate(output_1_data_trialAvg_list_dlpfc, axis=0)

# # output_0_data_trialAvg_acc = np.concatenate(output_0_data_trialAvg_list_acc, axis=0)
# # output_1_data_trialAvg_acc = np.concatenate(output_1_data_trialAvg_list_acc, axis=0)

# # output_0_data_trialAvg_parietal = np.concatenate(output_0_data_trialAvg_list_parietal, axis=0)
# # output_1_data_trialAvg_parietal = np.concatenate(output_1_data_trialAvg_list_parietal, axis=0)


# # # Concatenate time-averaged data across all subjects for each brain roi and condition
# # output_0_data_timeAvg_dlpfc = np.concatenate(output_0_data_timeAvg_list_dlpfc, axis=1)
# # output_1_data_timeAvg_dlpfc = np.concatenate(output_1_data_timeAvg_list_dlpfc, axis=1)

# # output_0_data_timeAvg_acc = np.concatenate(output_0_data_timeAvg_list_acc, axis=1)
# # output_1_data_timeAvg_acc = np.concatenate(output_1_data_timeAvg_list_acc, axis=1)

# # output_0_data_timeAvg_parietal = np.concatenate(output_0_data_timeAvg_list_parietal, axis=1)
# # output_1_data_timeAvg_parietal = np.concatenate(output_1_data_timeAvg_list_parietal, axis=1)


# # # Calculate mean and SEM across electrodes
# # overall_average_dlpfc_output_0 = np.nanmean(output_0_data_trialAvg_dlpfc, axis=0)
# # overall_sem_dlpfc_output_0 = np.std(output_0_data_trialAvg_dlpfc, axis=0, ddof=1) / np.sqrt(output_0_data_trialAvg_dlpfc.shape[0])
# # overall_average_dlpfc_output_1 = np.nanmean(output_1_data_trialAvg_dlpfc, axis=0)
# # overall_sem_dlpfc_output_1 = np.std(output_1_data_trialAvg_dlpfc, axis=0, ddof=1) / np.sqrt(output_1_data_trialAvg_dlpfc.shape[0])

# # overall_average_acc_output_0 = np.nanmean(output_0_data_trialAvg_acc, axis=0)
# # overall_sem_acc_output_0 = np.std(output_0_data_trialAvg_acc, axis=0, ddof=1) / np.sqrt(output_0_data_trialAvg_acc.shape[0])
# # overall_average_acc_output_1 = np.nanmean(output_1_data_trialAvg_acc, axis=0)
# # overall_sem_acc_output_1 = np.std(output_1_data_trialAvg_acc, axis=0, ddof=1) / np.sqrt(output_1_data_trialAvg_acc.shape[0])

# # overall_average_parietal_output_0 = np.nanmean(output_0_data_trialAvg_parietal, axis=0)
# # overall_sem_parietal_output_0 = np.std(output_0_data_trialAvg_parietal, axis=0, ddof=1) / np.sqrt(output_0_data_trialAvg_parietal.shape[0])
# # overall_average_parietal_output_1 = np.nanmean(output_1_data_trialAvg_parietal, axis=0)
# # overall_sem_parietal_output_1 = np.std(output_1_data_trialAvg_parietal, axis=0, ddof=1) / np.sqrt(output_1_data_trialAvg_parietal.shape[0])


# # # Run the time perm cluster test
# # mat_dlpfc = time_perm_cluster(output_0_data_trialAvg_dlpfc, output_1_data_trialAvg_dlpfc, 0.05, n_jobs=6)
# # mat_acc = time_perm_cluster(output_0_data_trialAvg_acc, output_1_data_trialAvg_acc, 0.05, n_jobs=6)
# # mat_parietal = time_perm_cluster(output_0_data_trialAvg_parietal, output_1_data_trialAvg_parietal, 0.05, n_jobs=6)

### do window stats  
use the time avg outputs from previous cell  
use fdr correction after comparing output 0 and output 1 for each electrode to get a p-values list  

DO A SHUFFLE INSTEAD OF PAIRED T-TEST AS OF 2/7/24


shuffle test (perm test). This basically time perm cluster but avg across time.

In [100]:
import numpy as np

def permutation_test(data_timeavg_output_0, data_timeavg_output_1, n_permutations=10000):
    """
    Perform a permutation test to compare two conditions.

    Parameters:
    - data_timeavg_output_0: Numpy array for condition 0.
    - data_timeavg_output_1: Numpy array for condition 1.
    - n_permutations: Number of permutations to perform.

    Returns:
    - p_value: P-value assessing the significance of the observed difference.
    """
    # Calculate the observed difference in means between the two conditions
    observed_diff = np.nanmean(data_timeavg_output_0) - np.nanmean(data_timeavg_output_1)
    
    # Combine the data from both conditions
    combined_data = np.hstack([data_timeavg_output_0, data_timeavg_output_1])
    
    # Initialize a variable to count how many times the permuted difference exceeds the observed difference
    count_extreme_values = 0
    
    for _ in range(n_permutations):
        # Shuffle the combined data
        np.random.shuffle(combined_data)
        
        # Split the shuffled data back into two new groups
        permuted_0 = combined_data[:len(data_timeavg_output_0)]
        permuted_1 = combined_data[len(data_timeavg_output_0):]
        
        # Calculate the mean difference for this permutation
        permuted_diff = np.nanmean(permuted_0) - np.nanmean(permuted_1)
        
        # Check if the permuted difference is as extreme as the observed difference
        if abs(permuted_diff) >= abs(observed_diff):
            count_extreme_values += 1
    
    # Calculate the p-value
    p_value = count_extreme_values / n_permutations
    
    return p_value


In [111]:
def perform_permutation_test_within_electrodes(data_0_list, data_1_list, n_permutations=10000):
    """
    Perform a permutation test for each electrode comparing two conditions across subjects.
    
    Parameters:
    - data_0_list: List of subject arrays from condition 0, each array is trials x electrodes.
    - data_1_list: List of subject arrays from condition 1, each array is trials x electrodes.
    - n_permutations: Number of permutations for the test.
    
    Returns:
    - p_values: A list of p-values for each electrode, across all subjects.
    """
    p_values = []

    # Ensure there is a corresponding condition 1 array for each condition 0 array
    if len(data_0_list) != len(data_1_list):
        raise ValueError("Mismatch in number of subjects between conditions")

    # Iterate through each subject's data arrays
    for idx, (data_0, data_1) in enumerate(zip(data_0_list, data_1_list)):
        print(f"Subject {idx} - Condition 0 shape: {data_0.shape}, Condition 1 shape: {data_1.shape}")

        # Check for matching electrode counts between conditions within a subject
        if data_0.shape[1] != data_1.shape[1]:
            raise ValueError(f"Electrode count mismatch in subject {idx}")

        n_electrodes_this_sub = data_0.shape[1]  # Number of electrodes for this subject

        # Perform the permutation test for each electrode in this subject
        for electrode_idx in range(n_electrodes_this_sub):  # Fix: use range(n_electrodes) to iterate correctly
            p_value = permutation_test(data_0[:, electrode_idx], data_1[:, electrode_idx], n_permutations)
            p_values.append(p_value)

    return p_values


In [116]:
def perform_permutation_test_across_electrodes(data_0_list, data_1_list, n_permutations=10000):
    """
    Perform a permutation test across electrodes comparing two conditions.
    
    Parameters:
    - data_0_list: List of arrays from condition 0, each array is trials x electrodes.
    - data_1_list: List of arrays from condition 1, each array is trials x electrodes.
    - n_permutations: Number of permutations for the test.
    
    Returns:
    - p_value: P-value from the permutation test.
    """
    # Aggregate data across electrodes
    data_0_aggregated = np.concatenate([np.nanmean(data, axis=0) for data in data_0_list])  # Average across trials to get a single value per electrode
    data_1_aggregated = np.concatenate([np.nanmean(data, axis=0) for data in data_1_list])  # though should I do avg across electrodes instead..?? Uhhhh. No, I think.
    
    # Perform the permutation test
    p_value = permutation_test(data_0_aggregated, data_1_aggregated, n_permutations)
    
    return p_value

do perm testing

In [127]:
# Assuming the functions perform_permutation_test_within_electrodes and perform_permutation_test_across_electrodes return lists of p-values
p_values = {}
rois = ['dlpfc', 'acc', 'parietal']
for roi in rois:
    # Initialize p_values[roi] as a dictionary
    p_values[roi] = {}

    # Perform the tests and store results
    p_values[roi]['within'] = perform_permutation_test_within_electrodes(output_0_data_timeAvg_firstHalfSecond_list[roi], output_1_data_timeAvg_firstHalfSecond_list[roi], n_permutations=10000)
    p_values[roi]['across'] = perform_permutation_test_across_electrodes(output_0_data_timeAvg_firstHalfSecond_list[roi], output_1_data_timeAvg_firstHalfSecond_list[roi], n_permutations=10000)

all_p_values = []
for roi in p_values:
    for test_type in p_values[roi]:
        p = p_values[roi][test_type]
        if isinstance(p, list):
            all_p_values.extend(p)
        else:  # Assume it's a single float value
            all_p_values.append(p)

# Apply FDR correction
_, adjusted_p_values = multipletests(all_p_values, alpha=0.05, method='fdr_bh')[:2]

# Incorporating adjusted p-values back into the structure is a bit more complex and depends on how you want to use them next


Subject 0 - Condition 0 shape: (121, 1), Condition 1 shape: (102, 1)
Subject 1 - Condition 0 shape: (110, 4), Condition 1 shape: (113, 4)
Subject 2 - Condition 0 shape: (118, 6), Condition 1 shape: (104, 6)
Subject 3 - Condition 0 shape: (105, 1), Condition 1 shape: (117, 1)
Subject 4 - Condition 0 shape: (109, 4), Condition 1 shape: (111, 4)
Subject 5 - Condition 0 shape: (111, 1), Condition 1 shape: (111, 1)
Subject 6 - Condition 0 shape: (117, 1), Condition 1 shape: (106, 1)
Subject 7 - Condition 0 shape: (110, 4), Condition 1 shape: (113, 4)
Subject 0 - Condition 0 shape: (118, 4), Condition 1 shape: (104, 4)
Subject 1 - Condition 0 shape: (117, 1), Condition 1 shape: (106, 1)
Subject 2 - Condition 0 shape: (110, 1), Condition 1 shape: (113, 1)
Subject 0 - Condition 0 shape: (121, 3), Condition 1 shape: (102, 3)
Subject 1 - Condition 0 shape: (110, 3), Condition 1 shape: (113, 3)
Subject 2 - Condition 0 shape: (109, 1), Condition 1 shape: (111, 1)
Subject 3 - Condition 0 shape: (11

In [129]:
p_values

{'dlpfc': {'within': [0.2972,
   0.2641,
   0.9632,
   0.5992,
   0.2678,
   0.0882,
   0.246,
   0.3544,
   0.8306,
   0.312,
   0.9271,
   0.7132,
   0.8788,
   0.2037,
   0.9557,
   0.3454,
   0.6933,
   0.5169,
   0.369,
   0.6368,
   0.6565,
   0.2463],
  'across': 0.9368},
 'acc': {'within': [0.407, 0.8218, 0.6427, 0.7348, 0.3698, 0.5994],
  'across': 0.6079},
 'parietal': {'within': [0.0769,
   0.306,
   0.9659,
   0.6881,
   0.138,
   0.0064,
   0.1829,
   0.579,
   0.8397,
   0.9703,
   0.9993,
   0.9642,
   0.9975],
  'across': 0.8878}}

In [136]:
electrode_mapping_per_roi

{'dlpfc': [('D0057', 'RPI14', 0),
  ('D0059', 'LMMF9', 1),
  ('D0059', 'LMMF11', 2),
  ('D0059', 'LMMF12', 3),
  ('D0059', 'LPSF16', 4),
  ('D0063', 'LMSF5', 5),
  ('D0063', 'LPSF12', 6),
  ('D0063', 'RAMF12', 7),
  ('D0063', 'RMMF13', 8),
  ('D0063', 'RMMF14', 9),
  ('D0063', 'RMMF10', 10),
  ('D0065', 'RASF14', 11),
  ('D0071', 'RIA11', 12),
  ('D0071', 'RIA12', 13),
  ('D0071', 'RIA14', 14),
  ('D0071', 'RIA16', 15),
  ('D0090', 'RIA12', 16),
  ('D0102', 'RFAM15', 17),
  ('D0103', 'LFAM8', 18),
  ('D0103', 'LFAM9', 19),
  ('D0103', 'LAI13', 20),
  ('D0103', 'LAI14', 21)],
 'acc': [('D0063', 'LMSF2', 0),
  ('D0063', 'LMSF4', 1),
  ('D0063', 'RMSF3', 2),
  ('D0063', 'RMSF4', 3),
  ('D0102', 'RFMM1', 4),
  ('D0103', 'LFAM3', 5)],
 'parietal': [('D0057', 'RPIP11', 0),
  ('D0057', 'RPIP12', 1),
  ('D0057', 'RPIP14', 2),
  ('D0059', 'LMPF11', 3),
  ('D0059', 'LMPF12', 4),
  ('D0059', 'LMPF14', 5),
  ('D0071', 'RTPS9', 6),
  ('D0077', 'ROAS14', 7),
  ('D0077', 'ROPS10', 8),
  ('D0077', 'RO

In [131]:
adjusted_pvals

array([0.95712941, 0.95712941, 0.9993    , 0.9993    , 0.95712941,
       0.95712941, 0.95712941, 0.95712941, 0.9993    , 0.95712941,
       0.9993    , 0.9993    , 0.9993    , 0.95712941, 0.9993    ,
       0.95712941, 0.9993    , 0.9993    , 0.95712941, 0.9993    ,
       0.9993    , 0.95712941, 0.9993    , 0.99488889, 0.9993    ,
       0.9993    , 0.9993    , 0.95712941, 0.9993    , 0.9993    ,
       0.95712941, 0.95712941, 0.9993    , 0.9993    , 0.95712941,
       0.2816    , 0.95712941, 0.9993    , 0.9993    , 0.9993    ,
       0.9993    , 0.9993    , 0.9993    , 0.9993    ])

### do anova  
this requires reloading in all four conditions (four this time cuz interaction contrasts).  
ONLY RUN THIS WHEN LOADING IN THE FOUR INTERACTION CONTRASTS RIGHT NOW.  
Integrate with other stats and plotting and stuff later.

untested 2/7

In [ ]:
# Assuming output_names contains all four conditions
output_data_trialAvg_lists = {output_name: {'dlpfc': [], 'acc': [], 'parietal': []} for output_name in output_names}
output_data_timeAvg_firstHalfSecond_lists = {output_name: {'dlpfc': [], 'acc': [], 'parietal': []} for output_name in output_names}
output_data_timeAvg_secondHalfSecond_lists = {output_name: {'dlpfc': [], 'acc': [], 'parietal': []} for output_name in output_names}
output_data_timeAvg_fullSecond_lists = {output_name: {'dlpfc': [], 'acc': [], 'parietal': []} for output_name in output_names}

for sub in subjects:
    for roi in ['dlpfc', 'acc', 'parietal']:
        for output_name in output_names:
            # Determine significant electrodes for the current ROI and subject
            sig_electrodes = sig_dlpfc_electrodes_per_subject.get(sub, []) if roi == 'dlpfc' else sig_acc_electrodes_per_subject.get(sub, []) if roi == 'acc' else sig_parietal_electrodes_per_subject.get(sub, [])
            
            if not sig_electrodes:  # Skip if no significant electrodes
                continue
            
            # Load trial-level data for the current condition and pick significant electrodes
            epochs = subjects_mne_objects[sub][output_name]['HG_ev1_rescaled'].copy().pick_channels(sig_electrodes)
            
            # Calculate averages for each time window
            trial_avg, time_avg_firstHalfSecond = filter_and_average_epochs(epochs, start_idx_firstHalfSecond, end_idx_firstHalfSecond)
            _, time_avg_secondHalfSecond = filter_and_average_epochs(epochs, start_idx_secondHalfSecond, end_idx_secondHalfSecond)
            _, time_avg_fullSecond = filter_and_average_epochs(epochs, start_idx_fullSecond, end_idx_fullSecond)
            
            # Append the results to their respective lists
            output_data_trialAvg_lists[output_name][roi].append(trial_avg)
            output_data_timeAvg_firstHalfSecond_lists[output_name][roi].append(time_avg_firstHalfSecond)
            output_data_timeAvg_secondHalfSecond_lists[output_name][roi].append(time_avg_secondHalfSecond)
            output_data_timeAvg_fullSecond_lists[output_name][roi].append(time_avg_fullSecond)


In [ ]:
# Example structure for organizing data
data_for_anova = []

# Assuming output_0_data_trialAvg_list and output_1_data_trialAvg_list are filled as per the conditions
for roi in ['dlpfc', 'acc', 'parietal']:
    for condition, trial_avg_data in [('Condition1', output_0_data_trialAvg_list[roi]), ('Condition2', output_1_data_trialAvg_list[roi])]:
        # Assume trial_avg_data is a list of arrays, each array corresponding to a subject
        for subject_index, subject_data in enumerate(trial_avg_data):
            subject_id = subjects[subject_index]  # Map back to subject ID
            for electrode_index, electrode_data in enumerate(subject_data):
                # Map electrode index back to electrode name if necessary
                # Here, electrode_data is the mean activity for this electrode under the current condition
                data_for_anova.append({
                    'SubjectID': subject_id,
                    'Electrode': electrode_mapping_per_roi[roi][electrode_index][1],  # Assuming mapping provides electrode names
                    'ROI': roi,
                    'Condition': condition,
                    'MeanActivity': np.mean(electrode_data)  # or directly use electrode_data if it's already a mean value
                })

# Convert to DataFrame
df_for_anova = pd.DataFrame(data_for_anova)

# Now df_for_anova is ready for further analysis, including ANOVA.


In [66]:
# import numpy as np
# from scipy import stats
# from statsmodels.stats.multitest import multipletests
# import numpy as np
# from scipy import stats
# from statsmodels.stats.multitest import multipletests

# def perform_analysis_and_fdr_correction(data_timeavg_output_0, data_timeavg_output_1):
#     """
#     Perform paired t-test for each electrode between two conditions and apply FDR correction,
#     ignoring NaN values and reporting the number of NaN pairs. Returns both adjusted and unadjusted p-values.

#     Parameters:
#     - data_timeavg_output_0: Numpy array for condition 0.
#     - data_timeavg_output_1: Numpy array for condition 1.

#     Returns:
#     - p_vals_adj: Adjusted p-values after FDR correction.
#     - unadj_p_vals: Original, unadjusted p-values from the t-tests.
#     - t_stats: T-statistics from the t-tests.
#     - nan_counts: Number of NaN pairs for each electrode, indicating missing data.
#     """
#     p_values = []
#     t_stats = []
#     nan_counts = []  # To store the count of NaN pairs for each electrode

#     # Ensure input data dimensions match
#     assert data_timeavg_output_0.shape == data_timeavg_output_1.shape, "Data arrays must have the same shape"

#     # Perform paired t-test for each electrode, ignoring NaNs
#     for i in range(data_timeavg_output_0.shape[1]):  # Iterate over electrodes
#         # Identify NaN pairs
#         nan_pairs = np.isnan(data_timeavg_output_0[:, i]) | np.isnan(data_timeavg_output_1[:, i])
#         nan_count = np.sum(nan_pairs)
#         nan_counts.append(nan_count)
        
#         # Remove NaN pairs for valid comparison
#         valid_indices = ~nan_pairs
#         if np.sum(valid_indices) > 1:  # Ensure there are at least two valid pairs for comparison
#             t_stat, p_value = stats.ttest_rel(data_timeavg_output_0[valid_indices, i],
#                                               data_timeavg_output_1[valid_indices, i])
#             t_stats.append(t_stat)
#             p_values.append(p_value)
#         else:
#             # Append NaNs if not enough data points for a valid test
#             t_stats.append(np.nan)
#             p_values.append(np.nan)

#     # Apply FDR correction
#     _, p_vals_adj, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

#     # Print out the number of NaN pairs for each electrode
#     for i, count in enumerate(nan_counts):
#         print(f"Electrode {i+1} had {count} NaN pairs excluded from analysis.")

#     return p_vals_adj, np.array(p_values), np.array(t_stats), np.array(nan_counts)

In [ ]:
# import numpy as np
# from scipy import stats
# from statsmodels.stats.multitest import multipletests
# import numpy as np
# from scipy import stats
# from statsmodels.stats.multitest import multipletests

# def perform_analysis_and_fdr_correction(data_timeavg_output_0, data_timeavg_output_1):
#     """
#     Perform paired t-test for each electrode between two conditions and apply FDR correction,
#     ignoring NaN values and reporting the number of NaN pairs. Returns both adjusted and unadjusted p-values.

#     Parameters:
#     - data_timeavg_output_0: Numpy array for condition 0.
#     - data_timeavg_output_1: Numpy array for condition 1.

#     Returns:
#     - p_vals_adj: Adjusted p-values after FDR correction.
#     - unadj_p_vals: Original, unadjusted p-values from the t-tests.
#     - t_stats: T-statistics from the t-tests.
#     - nan_counts: Number of NaN pairs for each electrode, indicating missing data.
#     """
#     p_values = []
#     t_stats = []
#     nan_counts = []  # To store the count of NaN pairs for each electrode

#     # Ensure input data dimensions match
#     assert data_timeavg_output_0.shape == data_timeavg_output_1.shape, "Data arrays must have the same shape"

#     # Perform paired t-test for each electrode, ignoring NaNs
#     for i in range(data_timeavg_output_0.shape[1]):  # Iterate over electrodes
#         # Identify NaN pairs
#         nan_pairs = np.isnan(data_timeavg_output_0[:, i]) | np.isnan(data_timeavg_output_1[:, i])
#         nan_count = np.sum(nan_pairs)
#         nan_counts.append(nan_count)
        
#         # Remove NaN pairs for valid comparison
#         valid_indices = ~nan_pairs
#         if np.sum(valid_indices) > 1:  # Ensure there are at least two valid pairs for comparison
#             t_stat, p_value = stats.ttest_rel(data_timeavg_output_0[valid_indices, i],
#                                               data_timeavg_output_1[valid_indices, i])
#             t_stats.append(t_stat)
#             p_values.append(p_value)
#         else:
#             # Append NaNs if not enough data points for a valid test
#             t_stats.append(np.nan)
#             p_values.append(np.nan)

#     # Apply FDR correction
#     _, p_vals_adj, _, _ = multipletests(p_values, alpha=0.05, method='fdr_bh')

#     # Print out the number of NaN pairs for each electrode
#     for i, count in enumerate(nan_counts):
#         print(f"Electrode {i+1} had {count} NaN pairs excluded from analysis.")

#     return p_vals_adj, np.array(p_values), np.array(t_stats), np.array(nan_counts)

In [155]:
# old code as of 2/6
# # Perform the analysis for DLPFC
# p_vals_adj_dlpfc, p_vals_dlpfc, t_stats_dlpfc, nan_pairs = perform_analysis_and_fdr_correction(output_0_data_timeAvg_dlpfc, output_1_data_timeAvg_dlpfc)

# # Perform the analysis for ACC
# p_vals_adj_acc, p_vals_acc, t_stats_acc, nan_pairs = perform_analysis_and_fdr_correction(output_0_data_timeAvg_acc, output_1_data_timeAvg_acc)

# # Perform the analysis for parietal
# p_vals_adj_parietal, p_vals_parietal, t_stats_parietal, nan_pairs = perform_analysis_and_fdr_correction(output_0_data_timeAvg_parietal, output_1_data_timeAvg_parietal)

Electrode 1 had 42 NaN pairs excluded from analysis.
Electrode 2 had 28 NaN pairs excluded from analysis.
Electrode 3 had 20 NaN pairs excluded from analysis.
Electrode 4 had 24 NaN pairs excluded from analysis.
Electrode 5 had 22 NaN pairs excluded from analysis.
Electrode 6 had 64 NaN pairs excluded from analysis.
Electrode 7 had 63 NaN pairs excluded from analysis.
Electrode 8 had 64 NaN pairs excluded from analysis.
Electrode 9 had 63 NaN pairs excluded from analysis.
Electrode 10 had 63 NaN pairs excluded from analysis.
Electrode 11 had 63 NaN pairs excluded from analysis.
Electrode 12 had 121 NaN pairs excluded from analysis.
Electrode 13 had 20 NaN pairs excluded from analysis.
Electrode 14 had 21 NaN pairs excluded from analysis.
Electrode 15 had 55 NaN pairs excluded from analysis.
Electrode 16 had 21 NaN pairs excluded from analysis.
Electrode 17 had 23 NaN pairs excluded from analysis.
Electrode 18 had 132 NaN pairs excluded from analysis.
Electrode 19 had 79 NaN pairs exclu

In [18]:
# # Define a dictionary to store the analysis results for easy access
# analysis_results = {}

# # Iterate over each roi and time window to perform the analysis
# for roi in ['dlpfc', 'acc', 'parietal']:
#     for time_window, concatenated_data in [
#         ('firstHalfSecond', concatenated_timeAvg_firstHalfSecond_data),
#         ('secondHalfSecond', concatenated_timeAvg_secondHalfSecond_data),
#         ('fullSecond', concatenated_timeAvg_fullSecond_data)
#     ]:
#         # Check if the roi has data for the current time window
#         if roi in concatenated_data:
#             output_0_data = concatenated_data[roi]['output_0']
#             output_1_data = concatenated_data[roi]['output_1']
            
#             # Perform the analysis
#             p_vals_adj, unadj_p_vals, t_stats, nan_counts = perform_analysis_and_fdr_correction(output_0_data, output_1_data)
            
#             # Store the results in the dictionary
#             result_key = f"{roi}_{time_window}"
#             analysis_results[result_key] = {
#                 'p_vals_adj': p_vals_adj,
#                 'unadj_p_vals': unadj_p_vals,
#                 't_stats': t_stats,
#                 'nan_counts': nan_counts
#             }

AssertionError: Data arrays must have the same shape

In [ ]:
# electrode_names_per_roi = {
#     'dlpfc': sig_dlpfc_electrodes_per_subject[sub],
#     'acc': sig_acc_electrodes_per_subject[sub],
#     'parietal': sig_parietal_electrodes_per_subject[sub]
# }

# # Adjust the script to print electrode names along with the results
# for roi in ['dlpfc', 'acc', 'parietal']:
#     print(f"Results for ROI: {roi}\n----------------------")
#     # Retrieve electrode names for the current ROI
#     electrode_names = electrode_names_per_roi.get(roi, [])
#     for time_window in ['firstHalfSecond', 'secondHalfSecond', 'fullSecond']:
#         key = f"{roi}_{time_window}"
#         if key in analysis_results:
#             results = analysis_results[key]
#             print(f"Time Window: {time_window}")
#             print(f"Electrode Names: {', '.join(electrode_names)}")
#             print(f"Adjusted p-values: {results['p_vals_adj']}")
#             print(f"Unadjusted p-values: {results['unadj_p_vals']}")
#             print(f"T-statistics: {results['t_stats']}")
#             print(f"NaN counts: {results['nan_counts']}\n")
#         else:
#             print(f"Time Window: {time_window} - No data\n")

Results for ROI: dlpfc
----------------------
Time Window: firstHalfSecond
Electrode Names: LFAM8, LFAM9, LAI13, LAI14
Adjusted p-values: [0.53085508 0.53085508 0.92322281 0.53085508 0.57536847 0.92322281
 0.57334206 0.98331877 0.53085508 0.92322281 0.53085508 0.63325222
 0.92322281 0.91270669 0.92322281 0.92322281 0.30677918 0.92322281
 0.94758699 0.57334206 0.92322281 0.53085508]
Unadjusted p-values: [0.16890843 0.15802348 0.78193704 0.07506122 0.26153112 0.60183525
 0.22189781 0.98331877 0.08868245 0.79391668 0.13339533 0.31662611
 0.8351666  0.49784001 0.76187148 0.83929346 0.01394451 0.73185498
 0.90451485 0.23454902 0.72857267 0.09995098]
T-statistics: [-1.38123857  1.41718879  0.27716292 -1.78946139 -1.12594555 -0.52280853
  1.22627248 -0.02094126 -1.71282162  0.26166446 -1.50851319  1.00635527
  0.20834742  0.67911875 -0.30351696 -0.20305881 -2.48052403 -0.34371307
 -0.12017143 -1.19372342 -0.34770072 -1.65574567]
NaN counts: [ 42  28  20  24  22  64  63  64  63  63  63 121  20

### plot and QC stats

check if no significant differences..

In [72]:
# Plotting
plt.figure(figsize=(10, 6))
plt.plot(time_perm_cluster_results['dlpfc'])
plt.xlabel('Timepoints')
plt.ylabel('Significance (0 or 1)')
plt.title('Permutation Test Significance Over Time')
plt.show()

KeyboardInterrupt: 

### plot dlpfc inc vs. con, avg across all subjects (or whatever conditions you wanna compare, just use the proper output names)

dlpfc  
use blue and orange for congruent and incongruent  
red and green for repeat and switch
# ONLY RUN EITHER SWITCHINESS OR CONGRUENCY!!!

### switchiness

is vs ir

In [50]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_dlpfc_ISvsIR_zscore_test.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0057_c = subjects_mne_objects['D0057'][output_names[0]]['HG_ev1_evoke_rescaled']

output_0_avg = overall_averages['dlpfc']['output_0']
output_1_avg = overall_averages['dlpfc']['output_1']
output_0_sem = overall_sems['dlpfc']['output_0']
output_1_sem = overall_sems['dlpfc']['output_1']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data
import matplotlib.pyplot as plt

# Determine the range of your data including SEM for better y-axis limits
min_val = min(output_0_avg.min() - output_0_sem.max(),
              output_1_avg.min() - output_1_sem.max())

max_val = max(output_0_avg.max() + output_0_sem.max(),
              output_1_avg.max() + output_1_sem.max())

# Optionally, add a small margin to the range
margin = (max_val - min_val) * 0.05  # 5% of the range as margin
min_val -= margin
max_val += margin

# Your existing plotting code
plt.figure(figsize=(10, 6))
plt.plot(times, output_0_avg, label='Average dlpfc inc repeat ', color='red')
plt.fill_between(times, output_0_avg - output_0_sem, 
                 output_0_avg + output_0_sem, alpha=0.3, color='red')
plt.plot(times, output_1_avg, label='Average dlpfc inc switch', color='green')
plt.fill_between(times, output_1_avg - output_1_sem, 
                 output_1_avg + output_1_sem, alpha=0.3, color='green')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average dlpfc Signal with Standard Error (inc switch vs inc repeat)')
plt.legend()

# Adjust the y-axis limits
plt.ylim([min_val, max_val])
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


KeyboardInterrupt: 

cs vs cr

In [77]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_dlpfc_CSvsCR_zscore_test.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0057_c = subjects_mne_objects['D0057'][output_names[0]]['HG_ev1_evoke_rescaled']

output_0_avg = overall_averages['dlpfc']['output_0']
output_1_avg = overall_averages['dlpfc']['output_1']
output_0_sem = overall_sems['dlpfc']['output_0']
output_1_sem = overall_sems['dlpfc']['output_1']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data
import matplotlib.pyplot as plt

# Determine the range of your data including SEM for better y-axis limits
min_val = min(output_0_avg.min() - output_0_sem.max(),
              output_1_avg.min() - output_1_sem.max())

max_val = max(output_0_avg.max() + output_0_sem.max(),
              output_1_avg.max() + output_1_sem.max())

# Optionally, add a small margin to the range
margin = (max_val - min_val) * 0.05  # 5% of the range as margin
min_val -= margin
max_val += margin

# Your existing plotting code
plt.figure(figsize=(10, 6))
plt.plot(times, output_0_avg, label='Average dlpfc con repeat ', color='red')
plt.fill_between(times, output_0_avg - output_0_sem, 
                 output_0_avg + output_0_sem, alpha=0.3, color='red')
plt.plot(times, output_1_avg, label='Average dlpfc con switch', color='green')
plt.fill_between(times, output_1_avg - output_1_sem, 
                 output_1_avg + output_1_sem, alpha=0.3, color='green')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average dlpfc Signal with Standard Error (con switch vs con repeat)')
plt.legend()

# Adjust the y-axis limits
plt.ylim([min_val, max_val])
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


KeyboardInterrupt: 

s vs r

In [35]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_dlpfc_switchiness_zscore_test.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0057_c = subjects_mne_objects['D0057'][output_names[0]]['HG_ev1_evoke_rescaled']

output_0_avg = overall_averages['dlpfc']['output_0']
output_1_avg = overall_averages['dlpfc']['output_1']
output_0_sem = overall_sems['dlpfc']['output_0']
output_1_sem = overall_sems['dlpfc']['output_1']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data
import matplotlib.pyplot as plt

# Determine the range of your data including SEM for better y-axis limits
min_val = min(output_0_avg.min() - output_0_sem.max(),
              output_1_avg.min() - output_1_sem.max())

max_val = max(output_0_avg.max() + output_0_sem.max(),
              output_1_avg.max() + output_1_sem.max())

# Optionally, add a small margin to the range
margin = (max_val - min_val) * 0.05  # 5% of the range as margin
min_val -= margin
max_val += margin

# Your existing plotting code
plt.figure(figsize=(10, 6))
plt.plot(times, output_0_avg, label='Average dlpfc repeat', color='red')
plt.fill_between(times, output_0_avg - output_0_sem, 
                 output_0_avg + output_0_sem, alpha=0.3, color='red')
plt.plot(times, output_1_avg, label='Average dlpfc switch', color='green')
plt.fill_between(times, output_1_avg - output_1_sem, 
                 output_1_avg + output_1_sem, alpha=0.3, color='green')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average dlpfc Signal with Standard Error (switch vs repeat)')
plt.legend()

# Adjust the y-axis limits
plt.ylim([min_val, max_val])
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


KeyboardInterrupt: 

### congruency  
add y lim to make the sem centered around the mean, can do this for other plots too if they look weird

inc-repeat vs con-repeat

In [73]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_dlpfc_IRvsCR_zscore_test.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

output_0_avg = overall_averages['dlpfc']['output_0']
output_1_avg = overall_averages['dlpfc']['output_1']
output_0_sem = overall_sems['dlpfc']['output_0']
output_1_sem = overall_sems['dlpfc']['output_1']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data
import matplotlib.pyplot as plt

# Determine the range of your data including SEM for better y-axis limits
min_val = min(output_0_avg.min() - output_0_sem.max(),
              output_1_avg.min() - output_1_sem.max())

max_val = max(output_0_avg.max() + output_0_sem.max(),
              output_1_avg.max() + output_1_sem.max())

# Optionally, add a small margin to the range
margin = (max_val - min_val) * 0.05  # 5% of the range as margin
min_val -= margin
max_val += margin

# Your existing plotting code
plt.figure(figsize=(10, 6))
plt.plot(times, output_0_avg, label='Average dlpfc con repeat', color='blue')
plt.fill_between(times, output_0_avg - output_0_sem, 
                 output_0_avg + output_0_sem, alpha=0.3, color='blue')
plt.plot(times, output_1_avg, label='Average dlpfc inc repeat', color='orange')
plt.fill_between(times, output_1_avg - output_1_sem, 
                 output_1_avg + output_1_sem, alpha=0.3, color='orange')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average dlpfc Signal with Standard Error (inc repeat vs con repeat)')
plt.legend()

# Adjust the y-axis limits
plt.ylim([min_val, max_val])
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


KeyboardInterrupt: 

inc-switch vs con-switch

In [68]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_dlpfc_ISvsCS_zscore_test.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

output_0_avg = overall_averages['dlpfc']['output_0']
output_1_avg = overall_averages['dlpfc']['output_1']
output_0_sem = overall_sems['dlpfc']['output_0']
output_1_sem = overall_sems['dlpfc']['output_1']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data
import matplotlib.pyplot as plt

# Determine the range of your data including SEM for better y-axis limits
min_val = min(output_0_avg.min() - output_0_sem.max(),
              output_1_avg.min() - output_1_sem.max())

max_val = max(output_0_avg.max() + output_0_sem.max(),
              output_1_avg.max() + output_1_sem.max())

# Optionally, add a small margin to the range
margin = (max_val - min_val) * 0.05  # 5% of the range as margin
min_val -= margin
max_val += margin

# Your existing plotting code
plt.figure(figsize=(10, 6))
plt.plot(times, output_0_avg, label='Average dlpfc con switch', color='blue')
plt.fill_between(times, output_0_avg - output_0_sem, 
                 output_0_avg + output_0_sem, alpha=0.3, color='blue')
plt.plot(times, output_1_avg, label='Average dlpfc inc switch', color='orange')
plt.fill_between(times, output_1_avg - output_1_sem, 
                 output_1_avg + output_1_sem, alpha=0.3, color='orange')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average dlpfc Signal with Standard Error (inc switch vs con switch)')
plt.legend()

# Adjust the y-axis limits
plt.ylim([min_val, max_val])
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


KeyboardInterrupt: 

inc vs con

In [180]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_dlpfc_congruency_zscore_test.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

output_0_avg = overall_averages['dlpfc']['output_0']
output_1_avg = overall_averages['dlpfc']['output_1']
output_0_sem = overall_sems['dlpfc']['output_0']
output_1_sem = overall_sems['dlpfc']['output_1']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data
import matplotlib.pyplot as plt

# Determine the range of your data including SEM for better y-axis limits
min_val = min(output_0_avg.min() - output_0_sem.max(),
              output_1_avg.min() - output_1_sem.max())

max_val = max(output_0_avg.max() + output_0_sem.max(),
              output_1_avg.max() + output_1_sem.max())

# Optionally, add a small margin to the range
margin = (max_val - min_val) * 0.05  # 5% of the range as margin
min_val -= margin
max_val += margin

# Your existing plotting code
plt.figure(figsize=(10, 6))
plt.plot(times, output_0_avg, label='Average dlpfc congruent', color='blue')
plt.fill_between(times, output_0_avg - output_0_sem, 
                 output_0_avg + output_0_sem, alpha=0.3, color='blue')
plt.plot(times, output_1_avg, label='Average dlpfc incongruent', color='orange')
plt.fill_between(times, output_1_avg - output_1_sem, 
                 output_1_avg + output_1_sem, alpha=0.3, color='orange')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average dlpfc Signal with Standard Error (inc vs con)')
plt.legend()

# Adjust the y-axis limits
plt.ylim([min_val, max_val])
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


KeyboardInterrupt: 

acc

switchiness

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_acc_switchRepeat_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_acc_output_0, label='Average acc repeat', color='red')
plt.fill_between(times, overall_average_acc_output_0 - overall_sem_acc_output_0, 
                 overall_average_acc_output_0 + overall_sem_acc_output_0, alpha=0.3, color='red')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_acc_output_1, label='Average acc switch', color='green')
plt.fill_between(times, overall_average_acc_output_1 - overall_sem_acc_output_1, 
                 overall_average_acc_output_1 + overall_sem_acc_output_1, alpha=0.3, color='green')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average acc Signal with Standard Error (switch vs repeat)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


congruency

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_acc_congruency_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_acc_output_0, label='Average acc congruent', color='blue')
plt.fill_between(times, overall_average_acc_output_0 - overall_sem_acc_output_0, 
                 overall_average_acc_output_0 + overall_sem_acc_output_0, alpha=0.3, color='blue')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_acc_output_1, label='Average acc incongruent', color='orange')
plt.fill_between(times, overall_average_acc_output_1 - overall_sem_acc_output_1, 
                 overall_average_acc_output_1 + overall_sem_acc_output_1, alpha=0.3, color='orange')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average acc Signal with Standard Error (inc vs con)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


parietal

switchiness

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_parietal_switchRepeat_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_parietal_output_0, label='Average parietal repeat', color='red')
plt.fill_between(times, overall_average_parietal_output_0 - overall_sem_parietal_output_0, 
                 overall_average_parietal_output_0 + overall_sem_parietal_output_0, alpha=0.3, color='red')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_parietal_output_1, label='Average parietal switch', color='green')
plt.fill_between(times, overall_average_parietal_output_1 - overall_sem_parietal_output_1, 
                 overall_average_parietal_output_1 + overall_sem_parietal_output_1, alpha=0.3, color='green')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average parietal Signal with Standard Error (switch vs repeat)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


congruency

In [ ]:
# Determine LAB_root based on the operating system
HOME = os.path.expanduser("~")
LAB_root = os.path.join(HOME, "Box", "CoganLab") if os.name == 'nt' else os.path.join(HOME, "Library", "CloudStorage", "Box-Box", "CoganLab")

# Get data layout
layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')
save_path = os.path.join(save_dir, 'avg_parietal_congruency_zscore.png')

plt.figure(figsize=(10, 6))

HG_ev1_evoke_rescaled_D0063_c = subjects_mne_objects['D0063'][output_names[0]]['HG_ev1_evoke_rescaled']

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data

# Plot overall_average_acc of the 0th output with SEM shading
plt.plot(times, overall_average_parietal_output_0, label='Average parietal congruent', color='blue')
plt.fill_between(times, overall_average_parietal_output_0 - overall_sem_parietal_output_0, 
                 overall_average_parietal_output_0 + overall_sem_parietal_output_0, alpha=0.3, color='blue')

# Plot overall_average_dlpfc of the 1st output with SEM shading
plt.plot(times, overall_average_parietal_output_1, label='Average parietal incongruent', color='orange')
plt.fill_between(times, overall_average_parietal_output_1 - overall_sem_parietal_output_1, 
                 overall_average_parietal_output_1 + overall_sem_parietal_output_1, alpha=0.3, color='orange')

plt.xlabel('Time (s)')
plt.ylabel('Z-score')
plt.title('Average parietal Signal with Standard Error (inc vs con)')
plt.legend()
plt.savefig(save_path)  # Modify save_dir as necessary
plt.show()


### plot individual electrodes

In [78]:
import matplotlib.pyplot as plt
import numpy as np

LAB_root = None
channels = None
full_trial_base = False


if LAB_root is None:
    HOME = os.path.expanduser("~")
    if os.name == 'nt':  # windows
        LAB_root = os.path.join(HOME, "Box", "CoganLab")
    else:  # mac
        LAB_root = os.path.join(HOME, "Library", "CloudStorage", "Box-Box",
                                "CoganLab")

layout = get_data(task, root=LAB_root)
save_dir = os.path.join(layout.root, 'derivatives', 'freqFilt', 'figs')

# Use the times from your evoked data (assuming these are representative for all subjects)
times = HG_ev1_evoke_rescaled_D0057_c.times  # Modify as needed to match your data

firstColor, secondColor= 'red', 'green' #switchiness
# firstColor, secondColor= 'blue', 'orange' #congruency

def plot_electrodes_grid(electrodes_data, grid_num, save_dir, roi, output_names, times, firstColor, secondColor):
    fig, axes = plt.subplots(4, 4, figsize=(20, 12))  # Adjust figure size as needed
    axes = axes.flatten()  # Flatten the axes array for easy indexing

    for i, (data, sub, electrode) in enumerate(electrodes_data):
        ax = axes[i]
        ax.plot(times, data['output_0'], label=f'{roi} {output_names[0]}', color=firstColor)
        ax.fill_between(times, 
                        data['output_0'] - np.std(data['output_0'], ddof=1) / np.sqrt(len(data['output_0'])),
                        data['output_0'] + np.std(data['output_0'], ddof=1) / np.sqrt(len(data['output_0'])), alpha=0.3)
        ax.plot(times, data['output_1'], label=f'{roi} {output_names[1]}', color=secondColor)
        ax.fill_between(times, 
                        data['output_1'] - np.std(data['output_1'], ddof=1) / np.sqrt(len(data['output_1'])),
                        data['output_1'] + np.std(data['output_1'], ddof=1) / np.sqrt(len(data['output_1'])), alpha=0.3)
        ax.set_title(f'Subject {sub}, Electrode {electrode}')
        ax.set_xlabel('Time (s)')
        ax.set_ylabel('Z-score')

    # Create the legend at the top center of the figure
    handles, labels = ax.get_legend_handles_labels()  # Get handles and labels from the last subplot
    fig.legend(handles, labels, loc='lower center', ncol=2)

    plt.tight_layout()  # Adjust the layout to make room for the legend
    plt.savefig(os.path.join(save_dir, f'{roi}_{output_names[0]}_{output_names[1]}_electrodes_plot_grid_{grid_num+1}.png'))
    plt.close()

# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0
roi = 'dlpfc'

# DUDE MAKE THE SIG ELECTRODES PER SUBJECT INTO A DICTIONARY. Bad code is bad.
for sub in subjects:
    if sub in sig_dlpfc_electrodes_per_subject:
        for electrode in sig_dlpfc_electrodes_per_subject[sub]:
            electrode_data = {
                'output_0': concatenated_trialAvg_data[roi]['output_0'][electrode_counter],
                'output_1': concatenated_trialAvg_data[roi]['output_1'][electrode_counter]
            }
            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, save_dir, roi, output_names, times, firstColor, secondColor)
                
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'dlPFC', output_names, times, firstColor, secondColor)



acc

In [ ]:
# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0

for sub in subjects:
    if sub in sig_acc_electrodes_per_subject:
        for electrode in sig_acc_electrodes_per_subject[sub]:
            electrode_data = {
                'output_0': output_0_data_trialAvg_acc[electrode_counter],
                'output_1': output_1_data_trialAvg_acc[electrode_counter]
            }
            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'acc', output_names, times, firstColor, secondColor)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'acc', output_names, times, firstColor, secondColor)


parietal

In [ ]:
# Example Usage
electrodes_data = []
electrode_counter = 0
grid_size = 16  # Number of electrodes per grid
grid_num = 0

for sub in subjects:
    if sub in sig_parietal_electrodes_per_subject:
        for electrode in sig_parietal_electrodes_per_subject[sub]:
            electrode_data = {
                'output_0': output_0_data_trialAvg_parietal[electrode_counter],
                'output_1': output_1_data_trialAvg_parietal[electrode_counter]
            }
            electrodes_data.append((electrode_data, sub, electrode))
            electrode_counter += 1

            if len(electrodes_data) == grid_size:
                plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'parietal', output_names, times, firstColor, secondColor)
                electrodes_data = []  # Reset for the next grid
                grid_num += 1

# Plot remaining electrodes in the last grid
if electrodes_data:
    plot_electrodes_grid(electrodes_data, grid_num, save_dir, 'parietal', output_names, times, firstColor, secondColor)
